In [2]:
import ee
#ee.Authenticate()
ee.Initialize()

In [3]:
import geemap
Map = geemap.Map(center=[39.027884,-84.137946], zoom=12)
Map.add_basemap('SATELLITE') # Add Google Map
Map

Map(center=[39.027884, -84.137946], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBo…

In [5]:
#search parameters are set here
from datetime import date, timedelta
pnt_date = date(2019, 11, 4)
lat =  39.027884
lon = -84.137946
delta = timedelta(days = 10)

$ \alpha^\frac{1}{h} $

In [6]:
from sentinel_search import sentinel2Availability, ImageExist
p1 = ee.Geometry.Point([lon, lat])
#  search sentinel first
img = sentinel2Availability(ee,pnt_date, lat, lon, delta, True)
bands = ['B1','B2','B3','B4','B5','B7']
if ImageExist(img) == 'true':
    ac_date = ee.Date(img.get('system:time_start')).format('MM/dd/YYYY');
    print('Found an ' + ' image for date:', pnt_date, ' at date:', ac_date.getInfo())
    img1 = img.select(['B2','B3','B4','B8A','B11','B12'],['B1','B2','B3','B4','B5','B7'])
    
    #the band names are defined to match Landsat TM bands
    vis = {'bands': ['B3', 'B2', 'B1'],'min': [0, 0, 0], 'max': [1000,1000, 1000]} #reflectance * 10000
    Map.addLayer(img1, vis,'Sentinel')
else:
    print('No image was found, try another date')

Found an  image for date: 2019-11-04  at date: 11/01/2019


In [7]:
roi = ee.Geometry.Polygon(
        [[[-84.2676, 39.0968],
          [-84.2676, 38.9562],
          [-84.0172, 38.9562],
          [-84.0172, 39.0968]]])
image_studyarea = img1.clip(roi)

In [8]:
Map.addLayer(image_studyarea, vis,'Sentine2')

In [9]:
#compute NDVI
ndvi = image_studyarea.normalizedDifference((['B4', 'B3']))
palette = ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718',
               '74A901', '66A000', '529400', '3E8601', '207401', '056201',
               '004C00', '023B01', '012E01', '011D01', '011301']
Map.addLayer(ndvi, {'min': 0, 'max': 1, 'palette': palette}, 'NDVI')

In [10]:
task1 = ee.batch.Export.image.toDrive(image=ndvi,description="NDVI",fileNamePrefix="NDVI",fileFormat="GeoTIFF",scale=30, folder="anyFolder")
task1.start()
task1.status()

{'state': 'READY',
 'description': 'NDVI',
 'creation_timestamp_ms': 1624498456402,
 'update_timestamp_ms': 1624498456402,
 'start_timestamp_ms': 0,
 'task_type': 'EXPORT_IMAGE',
 'id': 'IINP3X4QSHHXFTJ5PPOQI4UZ',
 'name': 'projects/earthengine-legacy/operations/IINP3X4QSHHXFTJ5PPOQI4UZ'}

In [12]:
task1.status()

{'state': 'RUNNING',
 'description': 'NDVI',
 'creation_timestamp_ms': 1624498456402,
 'update_timestamp_ms': 1624498470192,
 'start_timestamp_ms': 1624498467335,
 'task_type': 'EXPORT_IMAGE',
 'attempt': 1,
 'id': 'IINP3X4QSHHXFTJ5PPOQI4UZ',
 'name': 'projects/earthengine-legacy/operations/IINP3X4QSHHXFTJ5PPOQI4UZ'}